## Sentinel-1 burned area identification 

### Service Definition

In [ ]:
service = dict([('title', 'Sentinel-1 burned area identification'),
                ('abstract', 'Sentinel-1 burned area identification'),
                ('id', 'ewf-satcen-03-03-01')])

### Runtime parameter definition

In [ ]:
polarization = dict([('id', 'polarization'),
                     ('title', 'Polarization'),
                     ('abstract', 'Polarization'),
                     ('value', 'VV')])

In [ ]:
aoi = dict([('id', 'aoi'),
              ('title', 'Area of interest'),
              ('abstract', 'Area of interest'),
              ('value', 'POLYGON ((149.1733316344 -34.3218325277, 150.3652377054 -34.2948655567, 150.4056772124 -35.2832818792, 149.1994838956 -35.3112545059, 149.1733316344 -34.3218325277))')])

In [ ]:
epsg_code = dict([('id', 'epsg'),
                  ('title', 'EPSG code'),
                  ('abstract', 'EPSG code (example: EPSG:32632)'),
                  ('value', 'EPSG:32755')])

In [ ]:
resolution = dict([('id', 'resolution'),
                  ('title', 'Resolution in meters (5.0 or 10.0)'),
                  ('abstract', 'Resolution in meters (5.0 or 10.0)'),
                  ('value', '10.0')])

In [ ]:
orbit_type = dict([('id', 'orbit_type'),
                   ('title', 'Orbit type, Restituted or Precise'),
                   ('abstract', 'Orbit type, Restituted or Precise'),
                   ('value', 'Precise')])

In [ ]:
cde = dict([('id', 'change_detection_expression'),
            ('title', 'Change detection expression'),
            ('abstract', 'Change detection expression (where [0] corresponds to the master, [1] to the slave)'),
            ('value', '([0] &gt; 0.0001) &amp;&amp; ([1] &gt; 0.0001) &amp;&amp; (abs(log10([0] / [1])) &gt; 1) &amp;&amp; ([0] &gt; 0.05 || [1] &gt; 0.05)')])

**Input identifiers**

In [ ]:
input_identifiers = ['S1A_IW_GRDH_1SDV_20191025T191551_20191025T191616_029619_035F6B_CC56',
                     'S1A_IW_GRDH_1SDV_20191212T191551_20191212T191616_030319_0377C8_5456']

**Input references**

In [ ]:
input_references = ['https://catalog.terradue.com/sentinel1/search?format=json&uid=S1A_IW_GRDH_1SDV_20191025T191551_20191025T191616_029619_035F6B_CC56',
                    'https://catalog.terradue.com/sentinel1/search?format=json&uid=S1A_IW_GRDH_1SDV_20191212T191551_20191212T191616_030319_0377C8_5456']

**Data path**

This path defines where the data is staged-in. 

In [ ]:
data_path = '/workspace/data'

### Workflow

#### Import the packages required for processing the data

In [ ]:
import os
import sys
sys.path.append('/application/notebook/libexec/') 
sys.path.append(os.getcwd())

from shapely.wkt import loads

from helpers import *

sys.path.append('/opt/OTB/lib/python')
sys.path.append('/opt/OTB/lib/libfftw3.so.3')
os.environ['OTB_APPLICATION_PATH'] = '/opt/OTB/lib/otb/applications'
os.environ['LD_LIBRARY_PATH'] = '/opt/OTB/lib'
os.environ['ITK_AUTOLOAD_PATH'] = '/opt/OTB/lib/otb/applications'
import shutil

import otbApplication

%load_ext autoreload
%autoreload 2

In [ ]:
products = get_metadata(input_references, data_path)

In [ ]:
products.sort_values(by='startdate',ascending=True,inplace=True)


In [ ]:
products

In [ ]:
try:
    aoi_wkt = bbox_to_wkt(aoi['value'])
except:
    aoi_wkt = aoi['value']

In [ ]:
aoi_wkt

In [ ]:
pre_process(products=products,
             aoi=aoi_wkt,
             utm_zone=epsg_code['value'],
             resolution=resolution['value'],
             polarization=polarization['value'], 
             orbit_type=orbit_type['value'],
             show_graph=True)

#### Create the stack with the master and slave

In [ ]:
create_stack(products)

In [ ]:
stack_bands = list_bands('stack.dim')

In [ ]:
stack_bands

#### Change detection

In [ ]:
#change_detection_expression = '({0} &gt; 0.0001) &amp;&amp; ({1} &gt; 0.0001) &amp;&amp; (abs(log10({0} / {1})) &gt; 1) &amp;&amp; ({0} &gt; 0.05 || {1} &gt; 0.05)'.format(stack_bands[0], stack_bands[1])

In [ ]:
change_detection_expression = cde['value'].replace('[','{').replace(']','}').format(stack_bands[0], stack_bands[1])

In [ ]:
if not '&amp;' in change_detection_expression:
    change_detection_expression = change_detection_expression.replace('&','&amp;').replace('>','&gt;').replace('<','&lt;')

In [ ]:
change_detection_expression

In [ ]:
try:
    change_detection('stack.dim', 'raw_change_detection.tif', change_detection_expression)
except Exception as e:
    raise ValueError('Error {} on change_detection_expression {}'.format(e,change_detection_expression))

#### Linear to dB

In [ ]:
[convert_dim('{}.dim'.format(n)) for n in products.identifier.values]

#### Match the intensities to the dimension of the change detection GeoTIFF

Fit the geometries of the master/slave intensities in dB GeoTIFFs to the change_detection GeoTIFF geometry 

In [ ]:
import otbApplication

for index in range(2):
    
    Superimpose = otbApplication.Registry.CreateApplication("Superimpose")


    Superimpose.SetParameterString('inr', 'raw_change_detection.tif')
    Superimpose.SetParameterString('inm', ['{}_db.tif'.format(n) for n in products.identifier.values][index])
    Superimpose.SetParameterString('out', ['{}_db_si.tif'.format(n) for n in products.identifier.values][index])

    Superimpose.ExecuteAndWriteOutput()
    

In [ ]:
['{}_db_si.tif'.format(n) for n in products.identifier.values]

#### RGB with intensities

Master/slave intensities in dB in red and blue channels. 

In [ ]:
r_channel = 'im1b1'
g_channel = '0'
b_channel = 'im2b1'

band_expressions = [r_channel, 
                    g_channel, 
                    b_channel]

In [ ]:
create_composite(['{}_db_si.tif'.format(n) for n in products.identifier.values],
                 'temp_red_blue_{}.tif'.format('rgb'), 
                 band_expressions)
    

#### RGB with intensities and change detection map

Prepare the base RGB composite for adding a next step with the change detection in red

* red channel: master
* green channel: master
* blue channel: slave

In [ ]:
r_channel = 'im1b1'
g_channel = 'im1b1'
b_channel = 'im2b1'

band_expressions = [r_channel, 
                    g_channel, 
                    b_channel]

In [ ]:
create_composite(['{}_db_si.tif'.format(n) for n in products.identifier.values],
                 'temp_red_green_blue_{}.tif'.format('rgb'), 
                 band_expressions)

Convert the change detection to byte, change detection values will become 255


In [ ]:
Convert = otbApplication.Registry.CreateApplication('Convert')

Convert.SetParameterString('in', 'raw_change_detection.tif')
Convert.SetParameterString('out','raw_change_detection_byte.tif')
Convert.SetParameterString('type', 'linear')
Convert.SetParameterString('channels', 'grayscale')
Convert.SetParameterInt('channels.grayscale.channel', 1)

Convert.ExecuteAndWriteOutput()


Create the final RGB replacing the values in the red channel with 255 where there's a change

In [ ]:
r_channel = 'im1b1 == 255 ? im1b1 : im2b1'
g_channel = 'im1b1 == 255 ? 0 : im2b2'
b_channel = 'im1b1 == 255 ? 0 : im2b3'

band_expressions = [r_channel, 
                    g_channel, 
                    b_channel]


BandMathX = otbApplication.Registry.CreateApplication("BandMathX")

BandMathX.SetParameterStringList('il', ['raw_change_detection_byte.tif', 'temp_red_green_blue_{}.tif'.format('rgb')])

BandMathX.SetParameterString('out', 'rgb_composite_intensity_cd.tif')
BandMathX.SetParameterOutputImagePixelType('out', otbApplication.ImagePixelType_uint8)
BandMathX.SetParameterString('exp', ';'.join(band_expressions))

BandMathX.ExecuteAndWriteOutput()

Create a meaningful output name for the results

In [ ]:
masterID = products.iloc[0].identifier
slaveID = products.iloc[1].identifier

In [ ]:
products['startdate'] = pd.to_datetime(products['startdate'])
products['enddate'] = pd.to_datetime(products['enddate'])

output_startdate = min(products['startdate'])
output_stopdate = max(products['enddate'])

date_format = '%Y%m%dT%H%m%S'


output_name = 'Burned_Area_S1_{0}_{1}'.format(masterID,
                                              slaveID)

#### Cloud optimized GeoTIFF

In [ ]:
output_name

In [ ]:
# red and blue intensities
cog('temp_red_blue_{}.tif'.format('rgb'), '{}-RB.rgb.tif'.format(output_name))

In [ ]:
# RGB with change detection
cog('rgb_composite_intensity_cd.tif', '{}-CD.rgb.tif'.format(output_name))

In [ ]:
out_base = '{}-CD.rgb.tif'.format(output_name)[0:-8]
out_mask = out_base + '.mask'
out_rbb = out_base +'.acd'

In [ ]:
out_mask

#### Create the  GeoTiff product with the Mask of changes (0-1 values) 

In [ ]:
create_mask('{}-CD.rgb.tif'.format(output_name), '{}.tif'.format(out_mask))

#### Create the RGB composite with the intensities as RED=Older_image, GREEN=Newer_image, BLUE=Newer_image

In [ ]:
create_rbb('{}-RB.rgb.tif'.format(output_name), '{}.tif'.format(out_rbb))

In [ ]:
for index, properties_file in enumerate(['result', 
                                         'stage-in',
                                         '{}-RB.rgb'.format(output_name),
                                         '{}-CD.rgb'.format(output_name),
                                          out_mask, 
                                          out_rbb]):

    date_format = '%Y-%m-%dT%H:%m:%SZ'
    
    if properties_file == 'result':
        
        title = 'Reproducibility notebook used for generating {0}'.format(output_name)
   
    elif properties_file == 'stage-in':

        title = 'Reproducibility stage-in notebook for Sentinel-1 data for generating {0}'.format(output_name)
        
    elif properties_file == out_mask:
        
        title = 'Mask of changes for {0} to {1}'.format(output_startdate.strftime(date_format),
                                                                  output_stopdate.strftime(date_format))
        
    elif properties_file == out_rbb:
        
        title = 'RGB composite for {0} to {1}'.format(output_startdate.strftime(date_format),
                                                                  output_stopdate.strftime(date_format))
    
    else:
      
        title = 'Change detection for {0} to {1}'.format(output_startdate.strftime(date_format),
                                                                  output_stopdate.strftime(date_format))
    
    with open(properties_file + '.properties', 'wb') as file:
        file.write('title={0}\n'.format(title))
        file.write('date={0}/{1}\n'.format(output_startdate.strftime(date_format),
                                           output_stopdate.strftime(date_format)))
        
        if index > 1:
            file.write('geometry={0}'.format(get_image_wkt(properties_file + '.tif')))

In [ ]:
os.remove('raw_change_detection_byte.tif')
os.remove('raw_change_detection.tif')
os.remove('temp_red_green_blue_rgb.tif')

In [ ]:
for index in range(2):
    #os.remove(['{}.tif'.format(n) for n in products.identifier.values][index])
    os.remove(['{}_db.tif'.format(n) for n in products.identifier.values][index])
    os.remove(['{}_db_si.tif'.format(n) for n in products.identifier.values][index])
    os.remove(['{}.dim'.format(n) for n in products.identifier.values][index])
    shutil.rmtree(['{}.data'.format(n) for n in products.identifier.values][index])

In [ ]:
shutil.rmtree('stack.data')
os.remove('stack.dim')

### License

This work is licenced under a [Attribution-ShareAlike 4.0 International License (CC BY-SA 4.0)](http://creativecommons.org/licenses/by-sa/4.0/) 

YOU ARE FREE TO:

* Share - copy and redistribute the material in any medium or format.
* Adapt - remix, transform, and built upon the material for any purpose, even commercially.

UNDER THE FOLLOWING TERMS:

* Attribution - You must give appropriate credit, provide a link to the license, and indicate if changes were made. You may do so in any reasonable manner, but not in any way that suggests the licensor endorses you or your use.
* ShareAlike - If you remix, transform, or build upon the material, you must distribute your contributions under the same license as the original.